# Calculate edit distance between two PKS architectures

Note: for reference here are two Python libraries for computing Levenshtein distance
* https://github.com/aflc/editdistance This one doesn't return the full matrix, or list of edit operations
* https://github.com/ztane/python-Levenshtein/ This one only works on strings

Here we will use the second one, and archive a unique unicode ID for each annotation configuration of each possible domain

**by Tyler W. H. Backman**

In [1]:
# Connect to ClusterCAD
import os, sys
sys.path.insert(0, '/clusterCAD')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "clusterCAD.settings")
import django
django.setup()
import pks.models

In [2]:
# os.system('pip3 install python-Levenshtein')

In [3]:
# Load libaries
import numpy as np
import Levenshtein # get via 'pip3 install python-Levenshtein'
import itertools

### Define functions to to convert ClusterCAD objects into a tuple of domains

In [4]:
def isActive(domain):
    # returns the catalytic activity of a domain as a boolean
    
    if type(domain) in {
                            pks.models.CAL, 
                            pks.models.AT, 
                            pks.models.TE, 
                            pks.models.KS, 
                            pks.models.ACP, 
                            pks.models.PCP,
                       }:
        return True # these domains are always active but don't contain the boolean internally
    else:
        return domain.active

def moduleToTuple(thisModule, inactives=True):
    # export a PKS module as a list of text strings
    if inactives:
        domains = thisModule.domains()
    else:
        domains = filter(lambda d: isActive(d), thisModule.domains())
    return tuple(repr(domain) + '_' + str(domain) for domain in domains)

### Get two PKSs from ClusterCAD to compare

In [5]:
# Test moduleToTuple by getting the Coronafacic acid synthase architecture
pks1 = pks.models.Cluster.objects.get(mibigAccession='BGC0000041.1')
pks1architecture = [domainText
                   for subunit in pks1.subunits()
                   for module in subunit.modules()
                   for domainText in moduleToTuple(module, inactives=False)]
pks1architecture

['AT_substrate cyclopentene, loading',
 'ACP_domain',
 'KS_domain',
 'AT_substrate emal, non-loading',
 'DH_active',
 'ER_active',
 'KR_type B1, active',
 'ACP_domain',
 'KS_domain',
 'AT_substrate mal, non-loading',
 'ACP_domain',
 'TE_non-cyclic']

In [6]:
# get Chlorizidine A architecture
pks2 = pks.models.Cluster.objects.get(mibigAccession='BGC0001172.1')
pks2architecture = [domainText
                   for subunit in pks2.subunits()
                   for module in subunit.modules()
                   for domainText in moduleToTuple(module, inactives=False)]
pks2architecture

['KS_domain',
 'AT_substrate DCP, loading',
 'ACP_domain',
 'KS_domain',
 'AT_substrate DCP, non-loading',
 'DH_active',
 'KR_type C1, active',
 'ACP_domain',
 'KS_domain',
 'AT_substrate mal, non-loading',
 'ACP_domain']

## Decompose into unicode string to test Levenshtein package

editops output:
The result is a list of triples (operation, spos, dpos), where operation is one of 'replace', 'insert', or 'delete'; spos and dpos are position of characters in the first (source) and the second (destination) strings.

transforming source_string to destination_string

In [7]:
# Create lookup table with a unique unicode char for each domain

target = pks1architecture.copy()
source = pks2architecture.copy()
uniqueDomains = set(source)
uniqueDomains.update(target)
lookupTable = {name: chr(id) for id, name in enumerate(uniqueDomains)}
lookupTable

{'ACP_domain': '\x00',
 'DH_active': '\x01',
 'KR_type B1, active': '\x02',
 'AT_substrate mal, non-loading': '\x03',
 'AT_substrate DCP, loading': '\x04',
 'AT_substrate cyclopentene, loading': '\x05',
 'AT_substrate DCP, non-loading': '\x06',
 'KS_domain': '\x07',
 'AT_substrate emal, non-loading': '\x08',
 'ER_active': '\t',
 'TE_non-cyclic': '\n',
 'KR_type C1, active': '\x0b'}

In [8]:
# Convert PKS designs into unicode strings

sourceString = ''.join([lookupTable[k] for k in source])
targetString = ''.join([lookupTable[k] for k in target])

In [9]:
# Compute string distance

Levenshtein.distance(sourceString, targetString)

6

In [10]:
# Show machine readable edit operations
editOps = Levenshtein.editops(sourceString, targetString)
editOps

[('delete', 0, 0),
 ('replace', 1, 0),
 ('replace', 4, 3),
 ('insert', 6, 5),
 ('replace', 6, 6),
 ('insert', 11, 11)]

In [11]:
# Print human readable edit operations 
for editOp in editOps:
    if editOp[0] == 'delete':
        print('delete %s(%s) from source' % (editOp[1]+1, source[editOp[1]]))
    if editOp[0] == 'replace':
        print('replace %s(%s) from source with %s(%s) from target' % (editOp[1]+1, source[editOp[1]], editOp[2]+1, target[editOp[2]]))
    if editOp[0] == 'insert':
            print('insert %s(%s) from target after element %s(%s) in source' % (editOp[2]+1, target[editOp[2]], editOp[1], source[editOp[1]-1]))

delete 1(KS_domain) from source
replace 2(AT_substrate DCP, loading) from source with 1(AT_substrate cyclopentene, loading) from target
replace 5(AT_substrate DCP, non-loading) from source with 4(AT_substrate emal, non-loading) from target
insert 6(ER_active) from target after element 6(DH_active) in source
replace 7(KR_type C1, active) from source with 7(KR_type B1, active) from target
insert 12(TE_non-cyclic) from target after element 11(ACP_domain) in source


## Connect to ClusterCAD and create a searchable database of all PKS designs

In [12]:
# get all PKSs
allPKSs = pks.models.Cluster.objects.filter(reviewed=True)
len(allPKSs)

72

In [13]:
def extractPKSarch(pksObject, skipStart=True):
    # convert a PKS object into a list defining it's domains
    arch = []
    for subunit in pksObject.subunits():
        for module in subunit.modules():
            if skipStart and module.order == 0:
                continue
            for domainText in moduleToTuple(module, inactives=False):
                arch.append(domainText)
    return arch             

In [14]:
# create dict with designs for each PKS
allPKSDesigns = {pks.mibigAccession: extractPKSarch(pks) for pks in allPKSs}

In [15]:
# create a set of all unique domains
allUniqueDomains = set(itertools.chain.from_iterable(allPKSDesigns.values()))
len(allUniqueDomains)

26

In [16]:
# create lookup table assigning a unique unicode to each domain
lookupTable = {name: chr(id) for id, name in enumerate(allUniqueDomains)}
lookupTable

{'ACP_domain': '\x00',
 'AT_substrate hexmal, non-loading': '\x01',
 'KR_type B, active': '\x02',
 'AT_substrate isobutmal, non-loading': '\x03',
 'ER_active': '\x04',
 'AT_substrate butmal, non-loading': '\x05',
 'KR_type A1, active': '\x06',
 'AT_substrate mal, non-loading': '\x07',
 'PCP_domain': '\x08',
 'KR_type U, active': '\t',
 'AT_substrate mxmal_ACP, non-loading': '\n',
 'AT_substrate hmal, non-loading': '\x0b',
 'KR_type B2, active': '\x0c',
 'KS_domain': '\r',
 'KR_type A, active': '\x0e',
 'AT_substrate mmal, non-loading, iterations 3': '\x0f',
 'AT_substrate emal, non-loading': '\x10',
 'AT_substrate mmal, non-loading': '\x11',
 'KR_type C2, active': '\x12',
 'DH_active': '\x13',
 'KR_type B1, active': '\x14',
 'AT_substrate DCP, non-loading': '\x15',
 'KR_type A2, active': '\x16',
 'TE_cyclic': '\x17',
 'TE_non-cyclic': '\x18',
 'KR_type C1, active': '\x19'}

In [17]:
# create a dict with strings for each PKS design
pksDesignStrings = {key: ''.join([lookupTable[k] for k in value]) for key, value in allPKSDesigns.items()}

In [18]:
pksDesignStrings

{'BGC0000015.1': '\r\x11\x16\x00\r\x11\x06\x00\r\x07\x13\t\x00\r\x07\x13\x14\x00\r\x07\x13\x14\x00\r\x07\x13\x14\x00\r\x07\x13\x14\x00\r\x07\x13\x14\x00\r\x07\x13\x14\x00\r\x07\x06\x00\r\x11\x16\x00\r\x07\x06\x00\r\x07\x19\x00\r\x07\x14\x00\r\x07\x00\r\x07\x13\x04\x14\x00\r\x07\x00\r\x07\x14\x00\x17',
 'BGC0000036.1': '\r\x11\x13\x14\x00\r\x07\x13\x14\x00\r\x07\x13\x14\x00\r\x07\x13\x04\x14\x00\r\x07\x13\x04\x14\x00\r\x07\x13\x14\x00\r\x07\x13\t\x00\r\x07\x14\x00\r\x07\x13\x04\x14\x00\r\x11\x13\x14\x00\r\x07\x19\x00',
 'BGC0000001.1': '\r\x07\x13\t\x00\r\x07\x13\x14\x00\r\x07\x13\t\x00\r\x11\x13\x19\x00\r\x11\x13\x04\x14\x00\r\x07\x00',
 'BGC0000018.1': '\r\x11\x06\x00\r\x11\x13\x14\x00\r\x07\x13\x14\x00\r\x11\x12\x00\r\x10\x13\x04\x14\x00\r\x11\x06\x00\r\x07\x06\x00\x17',
 'BGC0000031.1': '\r\x07\x14\x00\r\x07\x13\x14\x00\r\x11\x13\x14\x00\r\x11\x06\x00\r\x0f\x13\x04\x14\x00\r\x07\x14\x00\x17',
 'BGC0000032.1': '\r\x11\x14\x00\r\x11\x13\x04\x14\x00\r\x07\x19\x00\r\x11\x13\x04\x14\x00\

In [19]:
def alignAll(query, tryTruncations=False):
    # align query against all PKSs and return a sorted tuple of (mibig, editDistance)
    results = []
    for key, value in pksDesignStrings.items():
        bestResult = (key, Levenshtein.distance(query, value), 0)

        if tryTruncations:
            for truncation in range(1,len(value)):
                newResult = (key, Levenshtein.distance(query, value[0:-truncation]), truncation)
                if newResult[1] < bestResult[1]:
                    bestResult = newResult
        
        results.append(bestResult)
        
    return sorted(results, key=lambda x: x[1])

alignAll(pksDesignStrings['BGC0000058.1'])

[('BGC0000058.1', 0, 0),
 ('BGC0000028.1', 18, 0),
 ('BGC0000126.1', 18, 0),
 ('BGC0001004.1', 18, 0),
 ('BGC0000036.1', 19, 0),
 ('BGC0000109.1', 19, 0),
 ('BGC0000105.1', 20, 0),
 ('BGC0000084.1', 21, 0),
 ('BGC0000106.1', 22, 0),
 ('BGC0000135.1', 22, 0),
 ('BGC0000100.1', 23, 0),
 ('BGC0000093.1', 23, 0),
 ('BGC0000148.1', 23, 0),
 ('BGC0001012.1', 23, 0),
 ('BGC0001359.1', 23, 0),
 ('BGC0000103.1', 23, 0),
 ('BGC0000136.1', 24, 0),
 ('BGC0000158.1', 24, 0),
 ('BGC0001065.1', 24, 0),
 ('BGC0000157.1', 25, 0),
 ('BGC0000994.1', 25, 0),
 ('BGC0000165.1', 25, 0),
 ('BGC0000087.1', 26, 0),
 ('BGC0000086.1', 26, 0),
 ('BGC0000114.1', 26, 0),
 ('BGC0000059.1', 26, 0),
 ('BGC0000075.1', 27, 0),
 ('BGC0000050.1', 27, 0),
 ('BGC0001119.1', 27, 0),
 ('BGC0001287.1', 27, 0),
 ('BGC0001003.1', 28, 0),
 ('BGC0001040.1', 28, 0),
 ('BGC0000073.1', 29, 0),
 ('BGC0001396.1', 30, 0),
 ('BGC0000018.1', 31, 0),
 ('BGC0000020.1', 31, 0),
 ('BGC0000066.1', 31, 0),
 ('BGC0000067.1', 31, 0),
 ('BGC0000074

## Perform an alignment for a new design

In [20]:
# for this design we know we won't find the TE or starting module in the right place, so we'll align just modules 1 and 2

queryDesign = [
    # 'KS_domain',
    # 'AT_substrate amal, loading',
    # 'ACP_domain',
    'KS_domain'
    'AT_substrate mal, non-loading',
    'DH_active',
    'ER_active',
    'KR_type B1, active',
    'ACP_domain',    
    'AT_substrate allylmal, non-loading',
    'DH_active',
    'ER_active',
    'KR_type B1, active',
    'ACP_domain', 
    # 'TE_cyclic',
]

In [21]:
# convert query design into a text string
queryString = ''
newCharCounter = len(lookupTable) # start with new chars after end of lookup table
for domain in queryDesign:
    if domain in lookupTable.keys():
        queryString = queryString + lookupTable[domain]
    else:
        newCharCounter += 1
        queryString = queryString + chr(newCharCounter)
queryString

'\x1b\x13\x04\x14\x00\x1c\x13\x04\x14\x00'

In [22]:
%%time
# perform alignment
topHits = alignAll(queryString, True)[0:10]
topHits

CPU times: user 10 ms, sys: 10 ms, total: 20 ms
Wall time: 20 ms


[('BGC0000066.1', 4, 24),
 ('BGC0000067.1', 4, 24),
 ('BGC0000074.1', 4, 24),
 ('BGC0000090.1', 4, 24),
 ('BGC0000032.1', 5, 14),
 ('BGC0000087.1', 5, 40),
 ('BGC0000086.1', 5, 40),
 ('BGC0000093.1', 5, 46),
 ('BGC0000144.1', 5, 53),
 ('BGC0000036.1', 6, 46)]

In [23]:
# print architeture of top hits
for hit in topHits:
    print(hit[0])
    print('truncation: ' + str(hit[2]))
    print(allPKSDesigns[hit[0]][0:-hit[2]])

BGC0000066.1
truncation: 24
['KS_domain', 'AT_substrate mmal, non-loading', 'DH_active', 'ER_active', 'KR_type B1, active', 'ACP_domain', 'KS_domain', 'AT_substrate mxmal_ACP, non-loading', 'DH_active', 'ER_active', 'KR_type B1, active', 'ACP_domain']
BGC0000067.1
truncation: 24
['KS_domain', 'AT_substrate mmal, non-loading', 'DH_active', 'ER_active', 'KR_type B1, active', 'ACP_domain', 'KS_domain', 'AT_substrate mxmal_ACP, non-loading', 'DH_active', 'ER_active', 'KR_type B1, active', 'ACP_domain']
BGC0000074.1
truncation: 24
['KS_domain', 'AT_substrate mmal, non-loading', 'DH_active', 'ER_active', 'KR_type B1, active', 'ACP_domain', 'KS_domain', 'AT_substrate mxmal_ACP, non-loading', 'DH_active', 'ER_active', 'KR_type B1, active', 'ACP_domain']
BGC0000090.1
truncation: 24
['KS_domain', 'AT_substrate mmal, non-loading', 'DH_active', 'ER_active', 'KR_type B1, active', 'ACP_domain', 'KS_domain', 'AT_substrate mxmal_ACP, non-loading', 'DH_active', 'ER_active', 'KR_type B1, active', 'ACP_d